# ETL 03 - Scripts Area Stage SQL - Geolocalizacao
    Objetivo: Popular a tabelas TOiSTG_CidadeGeoLocalizacao no banco DSTGOiMasterDados
    Desenvolvedor: Marcelo Muniz de Alencar

In [1]:
!pip install geopy # instalar geopy

ERROR: Invalid requirement: '#'



In [2]:
from geopy.geocoders import Nominatim
import pyodbc
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

C:\ProgramData\Anaconda3\lib\site-packages\setuptools\_importlib.py:23: UserWarning: `importlib-metadata` version is incompatible with `setuptools`.
This problem is likely to be solved by installing an updated version of `importlib-metadata`.
  warnings.warn(msg)  # Ensure a descriptive message is shown.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
connection = pyodbc.connect("Driver={SQL Server};Server=PCMARCELO\\SQLEXPRESS;Database=DSTG_OiMasterDados;Trusted_Connection=yes;") 
sql = ''' SELECT  NumCidade,
                  case when rtrim(ltrim(Estado)) <> ''
                        then Cidade + ', ' + Estado + ', '+ Pais
                        else Cidade + ', ' + Pais
                  end as Localizacao
          FROM OiMasterDados.TOiSTG_Cidade (NOLOCK) 
          ORDER BY NumCidade '''
df = pd.read_sql(sql,connection)

listaLocalizacao = df['Localizacao'].tolist() # criar lista
listaLocalizacao

OperationalError: ('08001', '[08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]SQL Server inexistente ou acesso negado. (17) (SQLDriverConnect); [08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionOpen (Connect()). (53)')

In [ ]:
# Initialize Nominatim API
geolocator = Nominatim(user_agent="MyApp")

cursor = connection.cursor()
cursor.execute('DELETE FROM [DSTG_OiMasterDados].[OiMasterDados].[TOiSTG_CidadeGeoLocalizacao]')
connection.commit()

for i in listaLocalizacao:
    try:
        location = geolocator.geocode(i)
        #print(i) 
        #print(str(location.latitude) + ',' + str(location.longitude))
        sqlInsert =  ''' INSERT INTO [OiMasterDados].[TOiSTG_CidadeGeoLocalizacao]
                               ([Localizacao]
                               ,[ValLatitude]
                               ,[ValLongitude])
                         VALUES
                           ( ''' + "'" + i + "'" + ', ' + str(location.latitude) + ', ' + str(location.longitude) + ')' 
        #print(sqlInsert)
        cursor.execute(sqlInsert)
        connection.commit()
    except:
        print("Erro na cidade " + i)

In [ ]:
sql2 = ''' SELECT  case when T1.[Cidade] = 'Montvideo' 
                            then 'Montevideo' 
                            else T1.[Cidade]
                    end as Localizacao
            FROM  [DSTG_OiMasterdados].[OiMasterDados].[TOiSTG_Cidade] AS T1 (NOLOCK) 
                        left outer join
                  [DSTG_OiMasterdados].[OiMasterDados].[TOiSTG_CidadeGeoLocalizacao] AS T2 (NOLOCK)
                        on (case when rtrim(ltrim(T1.Estado)) <> ''
                                    then T1.Cidade + ', ' + T1.Estado + ', '+ T1.Pais
                                    else T1.Cidade + ', ' + T1.Pais
                            end) = T2.Localizacao
            WHERE T2.Localizacao is null '''
df2 = pd.read_sql(sql2,connection)

listaLocalizacao2 = df2['Localizacao'].tolist() # criar lista
listaLocalizacao2

In [ ]:
# Initialize Nominatim API
geolocator = Nominatim(user_agent="MyApp")

for i in listaLocalizacao2:
    try:
        location2 = geolocator.geocode(i)
        #print(i) 
        #print(str(location2.latitude) + ',' + str(location2.longitude))
        sqlInsert2 =  ''' INSERT INTO [OiMasterDados].[TOiSTG_CidadeGeoLocalizacao]
                               ([Localizacao]
                               ,[ValLatitude]
                               ,[ValLongitude])
                         VALUES
                           ( ''' + "'" + i + "'" + ', ' + str(location2.latitude) + ', ' + str(location2.longitude) + ')' 
        #print(sqlInsert2)
        cursor.execute(sqlInsert2)
        connection.commit()
    except:
        print("Erro na cidade " + i)

## FIM